En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

In [13]:
file_path = "farmers-protest-tweets-2021-2-4.json"

## Consideraciones

En todos los casos abordados, se parte del supuesto de que los datos han pasado por una exhaustiva inspección de calidad previa. Esto implica que se encuentran en un estado óptimo, con todas las etiquetas correspondientes debidamente asignadas y sin ningún tipo de corrupción.

## Posibles mejoras y uso de GCP

Considerando los requerimientos del proyecto, una mejora significativa podría ser la adopción de herramientas ofrecidas por Google Cloud Platform (GCP). Si bien se han propuesto soluciones utilizando BigTable para el almacenamiento de datos semiestructurados, como los tweets, resulta pertinente señalar que esta elección podría no ser la más óptima.

Por ejemplo, los tweets contienen una variedad de metadatos y relaciones que podrían beneficiarse de un almacenamiento más estructurado, como el que proporciona BigQuery. Por lo tanto, una mejora potencial sería aprovechar BigQuery para el almacenamiento de la información, ya que está especialmente diseñado para manejar grandes volúmenes de datos y permite consultas rápidas y flexibles sobre ellos.

En cuanto al procesamiento de los datos, se podría seguir una lógica escalonada para lograr una mayor eficiencia:

1.  Definición de la consulta en BigQuery: Especificar la consulta necesaria para obtener los datos relevantes.
2.  Ejecución de la consulta desde Dataflow: Utilizar Dataflow para ejecutar la consulta definida en BigQuery y obtener los resultados de manera eficiente y escalable.
3.  Procesamiento de los resultados: Aplicar transformaciones y operaciones de procesamiento necesarias, como la extracción de emojis, la identificación de usuarios y fechas, sobre los datos obtenidos.
4.  Escritura de resultados: Si el procesamiento es un proceso recurrente, almacenar los resultados en una plataforma de almacenamiento de datos adecuada, como Cloud Storage o BigQuery.

Esta aproximación, además de aprovechar la potencia y la escalabilidad de las herramientas de GCP, también facilitaría la integración y el mantenimiento del sistema en el futuro.

# Q1_TIME

La optimización del tiempo de ejecución de q1_time se logró reduciendo significativamente el tiempo de procesamiento mediante la implementación de técnicas eficientes en el manejo de datos.

Para mejorar la eficiencia, se adoptó una estrategia más sofisticada que aprovechaba el poder del multiprocesamiento. En esta versión optimizada, se realizó una lectura de los núcleos disponibles para la ejecución del código, permitiendo así el procesamiento en paralelo. Esto significó dividir la lista de datos en lotes de igual tamaño para ejecutar la función de manera simultánea y reducir el tiempo total de ejecución.

La determinación del tamaño de estos lotes se realizó dividiendo la cantidad total de datos por la cantidad de núcleos disponibles para la ejecución. Si el tamaño del lote era menor que la cantidad de núcleos, se optó por utilizar un único lote para evitar el desperdicio de recursos.

Posteriormente, utilizando funciones lambda, se identificaron las 10 fechas con la mayor cantidad de tweets. A partir de estas fechas, se procedió a determinar los usuarios con la mayor cantidad de tweets en cada una de ellas.

In [14]:
from q1_time import q1_time
import time

tiempo_comienzo = time.time()
print(q1_time(file_path))
tiempo_final = time.time()
print(str(tiempo_final-tiempo_comienzo) + ' segundos')

[(datetime.date(2021, 2, 12), 'RanbirS00614606'), (datetime.date(2021, 2, 13), 'MaanDee08215437'), (datetime.date(2021, 2, 17), 'RaaJVinderkaur'), (datetime.date(2021, 2, 16), 'jot__b'), (datetime.date(2021, 2, 14), 'rebelpacifist'), (datetime.date(2021, 2, 18), 'neetuanjle_nitu'), (datetime.date(2021, 2, 15), 'jot__b'), (datetime.date(2021, 2, 20), 'MangalJ23056160'), (datetime.date(2021, 2, 23), 'Surrypuria'), (datetime.date(2021, 2, 19), 'Preetm91')]
3.8702001571655273 segundos


# Q1 MEMORY

La función q1_memory ha sido diseñada para optimizar el uso de memoria al procesar grandes conjuntos de datos contenidos en el archivo JSON.

Al abrir el archivo especificado en la ruta proporcionada, esta función procesa cada línea del archivo de forma individual utilizando el paquete ujson. Esta estrategia permite minimizar el uso de memoria al cargar y procesar los datos de forma incremental, en lugar de cargar todo el archivo en memoria de una vez.

Cada línea del archivo JSON se convierte en un objeto JSON utilizando la función ujson.loads(). A continuación, se extraen la fecha del tweet y el nombre de usuario asociado.

Para almacenar y procesar la actividad por fecha y usuario, se utiliza un diccionario predeterminado de defaultdict(Counter). Este diccionario permite contar eficientemente la actividad de cada usuario en cada fecha, utilizando la estructura de datos Counter para realizar un seguimiento de los conteos.

Una vez procesadas todas las líneas del archivo y almacenada la actividad por fecha y usuario, se utiliza la función heapq.nlargest() para encontrar las 10 fechas más activas de manera eficiente.

Para cada fecha seleccionada, se determina el usuario más activo y se agrega al resultado final como una tupla que contiene la fecha y el usuario más activo.


In [38]:
from q1_memory import q1_memory
from memory_profiler import memory_usage

inicio_memoria = memory_usage()[0]
print(q1_memory(file_path))
fin_memoria = memory_usage()[0]
print(str(fin_memoria-inicio_memoria) + ' Mb')

[(datetime.date(2021, 2, 12), 'RanbirS00614606'), (datetime.date(2021, 2, 13), 'MaanDee08215437'), (datetime.date(2021, 2, 17), 'RaaJVinderkaur'), (datetime.date(2021, 2, 16), 'jot__b'), (datetime.date(2021, 2, 14), 'rebelpacifist'), (datetime.date(2021, 2, 18), 'neetuanjle_nitu'), (datetime.date(2021, 2, 15), 'jot__b'), (datetime.date(2021, 2, 20), 'MangalJ23056160'), (datetime.date(2021, 2, 23), 'Surrypuria'), (datetime.date(2021, 2, 19), 'Preetm91')]
3.28125 Mb


# Q2 TIME

La función q2_memory ha sido desarrollada con el objetivo de contar la frecuencia de emojis en los tweets presentes en un archivo JSON, optimizando el uso de memoria durante este proceso.

Cada línea del archivo JSON se convierte en un objeto JSON utilizando la función ujson.loads(). A continuación, se extrae el contenido del tweet y se utilizan expresiones regulares Unicode para identificar y extraer los emojis presentes en el texto utilizando la función extract_emojis. El uso de expresiones regulares Unicode garantiza una identificación precisa de los emojis en el texto del tweet, lo que contribuye a la eficiencia general de la función. Por otro lado, el uso de expresiones regulares arrojo el mejor resultado en temas de tiempos de procesamiento en relacion a otras librerias como emoji o emojis.

Para almacenar y procesar la frecuencia de cada emoji, se utiliza un objeto Counter de la biblioteca estándar de Python. Este objeto permite contar eficientemente la frecuencia de cada emoji encontrado en los tweets.

Una vez procesadas todas las líneas del archivo y contada la frecuencia de los emojis, se utiliza el método most_common() del objeto Counter para encontrar los 10 emojis más utilizados de manera eficiente.

In [33]:
from q2_time import q2_time
import time

tiempo_comienzo = time.time()
print(q2_time(file_path))
tiempo_final = time.time()
print(str(tiempo_final-tiempo_comienzo) + ' segundos')

[('🙏', 7286), ('😂', 3072), ('🚜', 2972), ('✊', 2411), ('🌾', 2363), ('🇮', 2096), ('🇳', 2094), ('❤', 1779), ('🤣', 1668), ('👇', 1108)]
3.9355852603912354 segundos


# Q2_MEMORY

La función q2_memory se encarga de analizar un archivo JSON que contiene tweets para contar y devolver los 10 emojis más utilizados en el contenido de esos tweets.

Al abrir el archivo especificado en la ruta proporcionada, la función lee cada línea del archivo de manera secuencial utilizando la función ujson.loads() para cargar cada tweet como un objeto JSON, lo cual lo hace mas efectivo en terminos de memoria. Luego, extrae el contenido del tweet y utiliza la función extract_emojis() para obtener una lista de emojis presentes en ese contenido.

Para llevar un seguimiento de la frecuencia de cada emoji, se utiliza un contador (Counter()) que se actualiza con los emojis extraídos de cada tweet.

Finalmente, la función devuelve una lista de las 10 tuplas más comunes, donde cada tupla contiene un emoji y su frecuencia de aparición en los tweets.

In [39]:
from q2_memory    import q2_memory
from memory_profiler import memory_usage

inicio_memoria = memory_usage()[0]
print(q2_memory(file_path))
fin_memoria = memory_usage()[0]
print(str(fin_memoria-inicio_memoria) + ' Mb')

[('🙏', 7286), ('😂', 3072), ('🚜', 2972), ('✊', 2411), ('🌾', 2363), ('🏻', 2080), ('❤', 1779), ('🤣', 1668), ('🏽', 1218), ('👇', 1108)]
-1.23828125 Mb


# Q3_TIME

La función q3_time se encarga de procesar un archivo de texto para encontrar los usuarios más influyentes basados en menciones. Utiliza expresiones regulares para buscar y extraer las menciones de usuario en cada línea del archivo.

Al abrir el archivo especificado en la ruta proporcionada, la función itera sobre cada línea y busca todas las menciones de usuario utilizando la expresión regular @(\w+). Cada vez que se encuentra una mención, se incrementa el contador de menciones para ese usuario en un diccionario user_mentions, que almacena el recuento de menciones de cada usuario.

Una vez procesadas todas las líneas del archivo y recopiladas las menciones de usuario, se ordenan los usuarios por la cantidad de menciones en orden descendente utilizando la función sorted(). Luego, se devuelven las 10 tuplas más mencionadas, donde cada tupla contiene el nombre de usuario y la cantidad de menciones.

In [30]:
from q3_time import q3_time
import time

tiempo_comienzo = time.time()
print(q3_time(file_path))
tiempo_final = time.time()
print(str(tiempo_final-tiempo_comienzo) + ' segundos')

[('narendramodi', 5300), ('Kisanektamorcha', 4090), ('RakeshTikaitBKU', 3728), ('PMOIndia', 3114), ('RahulGandhi', 2704), ('GretaThunberg', 2622), ('DelhiPolice', 2254), ('rihanna', 2246), ('RaviSinghKA', 2242), ('INCIndia', 2236)]
1.2904407978057861 segundos


# Q3_MEMORY

La función q3_memory está diseñada para analizar un archivo de texto y encontrar los usuarios más influyentes basados en menciones. A diferencia de la función q3_time, esta versión optimiza el uso de memoria al procesar el archivo de una manera más eficiente.

Para lograr esto, la función utiliza un enfoque diferente para contar las menciones de usuario:

Conjunto de Usuarios Únicos: Se inicializa un conjunto unique_users para almacenar usuarios únicos con menciones. Este enfoque elimina la necesidad de almacenar cada mención individualmente, reduciendo así el uso de memoria al evitar duplicados.

Diccionario de Conteo de Menciones: Se utiliza un diccionario user_mentions para almacenar los recuentos de menciones de usuario. Cada vez que se encuentra una mención en el archivo, se incrementa el contador correspondiente en el diccionario.

Una vez procesado todo el archivo y recopiladas las menciones de usuario, se ordenan los usuarios por la cantidad de menciones en orden descendente utilizando la función sorted(). Luego, se devuelven las 10 tuplas más mencionadas, donde cada tupla contiene el nombre de usuario y la cantidad de menciones.

Este enfoque optimizado minimiza el uso de memoria al procesar el archivo de texto de manera más eficiente, lo que permite manejar archivos de gran tamaño sin agotar los recursos de memoria disponibles.

In [34]:
from q3_memory    import q3_memory
from memory_profiler import memory_usage

inicio_memoria = memory_usage()[0]
print(q3_memory(file_path))
fin_memoria = memory_usage()[0]
print(str(fin_memoria-inicio_memoria) + ' Mb')

[('narendramodi', 3364), ('PMOIndia', 2088), ('Kisanektamorcha', 1966), ('RakeshTikaitBKU', 1918), ('GretaThunberg', 1882), ('punyaab",', 1722), ('DelhiPolice', 1638), ('jazzyb', 1522), ('RahulGandhi', 1498), ('rihanna', 1496)]
0.51171875 Mb
